This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
# Grid parameters.
nh_min = 16
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
deriv = 'DD'
# RK = 4
order = 5

This cell contains the relevant switch information.

In [3]:
# TimeIntFunc, DiffFunc = TT.SolverSwitch(deriv, RK)

This cell instantiates the Grid and the Physics objects.

In [4]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)) + int(nh_min / refRatio))
omega.AddPatch(refRatio, finehalf)

This cell runs a test on the derivative operator on a cell-averaged polynomial which matches the given order.  
$P (x) = \sum_{k = 0}^{o} x^{k}$  
$<P (x)> = \frac{1}{\Delta x} \int_{x}^{x + \Delta x} P (x')$d$x'$  
$<P (x)> = \frac{1}{\Delta x} \sum_{k = 1}^{o} \frac{1}{k} [(x + \Delta x)^{k} - x^{k}]$  
$<\frac{d P (x)}{d x}> = \sum_{k = 1}^{o} [(x + \Delta x)^{k} - x^{k}]$  

In [5]:
TT.DerivPolyTest(omega, deriv, order, deriv = 0)

x:
[0.      0.0625  0.125   0.1875  0.25    0.3125  0.375   0.4375  0.5     0.53125 0.5625  0.59375 0.625   0.65625 0.6875  0.71875 0.75    0.78125 0.8125  0.84375 0.875   0.90625 0.9375  0.96875 1.     ]

Polynomial Function:
<p(x)> =
         6       5        4          3       2
0.1667 x + 0.2 x + 0.25 x + 0.3333 x + 0.5 x + 1 x
<p(x)> =
 [1.03261633 1.10388473 1.18570649 1.28052683 1.39144328 1.52232014 1.67790289 1.86393264 2.02616568 2.14835551 2.28241268 2.42951225 2.5909183  2.76798756 2.96217294 3.17502716 3.40820626 3.66347324 3.94270158 4.24787886 4.58111032 4.94462242 5.34076647 5.77202212]

Polynomial Derivative:
<dp(x)/dx> =
    5     4     3     2
1 x + 1 x + 1 x + 1 x + 1 x
Theoretical:
<dp(x)/dx> =
 [ 1.06666565  1.21897888  1.4058075   1.63667297  1.92292786  2.27775574  2.71617126  3.25502014  3.73204136  4.09391689  4.49200726  4.9291048   5.40811634  5.9320631   6.50408077  7.12741947  7.80544376  8.54163265  9.33957958 10.20299244 11.13569355 12.14161968 13.224822

In [6]:
coefs = 1. / (np.arange(order + 1) + 1)
coefs = np.append(1, coefs)[::-1]
P = np.poly1d(coefs) - 1
print(coefs)
print(P)
print('')

[0.16666667 0.2        0.25       0.33333333 0.5        1.         1.        ]
        6       5        4          3       2
0.1667 x + 0.2 x + 0.25 x + 0.3333 x + 0.5 x + 1 x



In [7]:
A = np.arange(10)
print(A)
print(A[:-1])
print('')

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8]

